In [0]:
from pyspark.sql.functions import col
from databricks.feature_store import FeatureStoreClient

In [0]:
# 1. Leer la base consolidada y hacer un muestreo
base_consolidada = spark.table("databricks_clase.prueba_schema.base_consolidada_v2")

In [0]:
# Ejemplo: tomar el 10% de los registros (puedes ajustar la fracción o utilizar .limit(n) si prefieres)
df_sample = base_consolidada.sample(fraction=0.1, seed=42)

In [0]:
df_sample.show(2)

+-------+----------+------------------+-------+-------------+-------------+-------------+----------------+------------+-------------+-----------+----------------+--------+------+-----------------+-----------------+------------------+------------------+------------------+-----------------+--------------------+------------------+------------------+--------------------+-------------+-------------+-------------+----------------+----------------+----------------+---------+-------------------+--------------------+
|periodo|id_cliente|tiempo_permanencia|flg_vip|incidencias_a|incidencias_b|tipo_producto|periodo_creacion|departamento|segmento_pago|      canal|segmento_cliente|crossell|  tasa|         monto_1m|         monto_2m|          monto_3m|          monto_4m|          monto_5m|         monto_6m|         cantidad_1m|       cantidad_2m|       cantidad_3m|         cantidad_6m|frecuencia_1m|frecuencia_2m|frecuencia_3m|ultima_compra_1m|ultima_compra_2m|ultima_compra_3m|flg_churn|        monto_

In [0]:
# 2. Leer la tabla de features del Feature Store (offline)
fs = FeatureStoreClient()
# Tabla de features mensuales (offline)
features_df1 = fs.read_table(name="databricks_clase.prueba_schema.base_consolidada_mensual_feats_v2")

# Tabla de features históricos (por ejemplo, con promedio y máximo)
features_df2 = fs.read_table(name="databricks_clase.prueba_schema.historico_feats")

# 3. Unir las dos tablas de Feature Store usando la clave primaria 'id_cliente'
features_joined = features_df1.join(features_df2, on="id_cliente", how="inner")

# 4. Unir el muestreo de la base consolidada con la unión de features
data_enriquecida = df_sample.select('id_cliente').join(features_joined, on="id_cliente", how="left")

# Convertir a Pandas para usar con scikit-learn (o trabajar en Spark si tu modelo lo permite)
data_pd = data_enriquecida.toPandas()

# Definir las columnas de features a utilizar para la inferencia (incluye features mensuales e históricos)
feature_list = [
    "frecuencia_1m", "cantidad_1m", "tasa", "ultima_compra_1m", "monto_total", 
    "avg_monto", "max_monto"
]
X_batch = data_pd[feature_list]

In [0]:
X_batch.head()

,frecuencia_1m,cantidad_1m,tasa,ultima_compra_1m,monto_total,avg_monto,max_monto
0,8.0,2.197225e+00,0.0280,29.0,49.792405,8.348998,8.996900
1,5.0,1.609438e+00,0.0344,29.0,44.895324,7.718552,8.531786
2,1.0,1.000000e-07,0.0280,10.0,-75.985308,-9.210340,4.605170
3,7.0,2.302585e+00,0.0344,24.0,-28.543193,-0.766155,7.398609
4,1.0,1.000000e-07,0.0280,5.0,44.886617,7.293134,8.515191


In [0]:
import mlflow
logged_model = 'runs:/8be5abd7b4254acbb00041830e627f47/modelo_simple'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [0]:
predicciones = loaded_model.predict(X_batch)
print("Predicciones batch:", predicciones)

Predicciones batch: [0. 0. 1. ... 0. 1. 0.]
